In [3]:
import itertools
from itertools import groupby
import pandas as pd
from pymongo import MongoClient

In [25]:
class Addition(object):
    def __init__(self):
        self.client = MongoClient('localhost', port=27017)
        self.db = self.client.train
        self.train = self.db.Train
        self.station = self.db.Station 

        self.train_df = pd.DataFrame(self.train.find({}, {'_id': 0}))
        self.train_df = self.train_df.drop_duplicates(subset='No', keep='first', inplace=False)

        self.station_df = pd.DataFrame(self.station.find({}, {'_id': 0}))
        self.station_df = self.station_df.drop_duplicates(subset='name', keep='first', inplace=False)

        self.type_dict = {'高速动车组': 'G', '普通动车组': 'D', '城际动车组': 'C', '快速': 'K',
                          '普快': '9', '直达特快': 'Z', '特快': 'T', '其他': 'O'}
                          
    def addTrainsOfStation(self):
        """
        将各个站点通过的列车加入Station表单
        :return: None
        """
        station = self.train_df.key.tolist()
        passby = list(zip(self.train_df.code, self.train_df.key))
        # 添加经过车站的列车
        trainsOfStation = {}
        for (code, names) in passby:
            for name in names:
                if name in trainsOfStation:
                    trainsOfStation[name].append(code)
                elif name != "":
                    trainsOfStation[name] = []
        for (name, codes) in trainsOfStation.items():
            #print(name, codes)              
            query = {"name":name}
            newvalues = { "$set": { "trains": codes } }
            self.station.update_one(query, newvalues)

In [26]:
addition = Addition()
addition.addTrainsOfStation()
#print(ls)